# Simpy in 10 minutes

Simple environments to start creating simulations.

In [2]:
# Importing libraries
import simpy

The first example is a car process. The car will alternately drive and park for a while. When it starts driving (or parking), it will print the current simulation time.

In [3]:
# Creating the generator
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [6]:
# Running the Model

env = simpy.Environment() # Creates the environment
env.process(car(env)) # Creates the car process
env.run(until=15) # Run the model

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


### Process Interaction

In [7]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))
            
            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
            
    def charge(self, duration):
        yield self.env.timeout(duration)

In [8]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


### Interrupting a process

In [12]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [15]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=25)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12
Start driving at 17
Start parking and charging at 19
Start driving at 24


In [1]:
for i in range(1,5):
    print(i**2)


1
4
9
16
